In [25]:
import scipy.io as sio
import os
import sys
import numpy as np
import pandas as pd
import time
import pickle
import torch
from tqdm import tqdm

In [21]:
def data_1Dto2D(data, Y=9, X=9):
    data_2D = np.zeros([Y, X])
    data_2D[0] = (0,        0,          0,          data[0],    0,          data[16],   0,          0,          0       )
    data_2D[1] = (0,        0,          0,          data[1],    0,          data[17],   0,          0,          0       )
    data_2D[2] = (data[3],  0,          data[2],    0,          data[18],   0,          data[19],   0,          data[20])
    data_2D[3] = (0,        data[4],    0,          data[5],    0,          data[22],   0,          data[21],   0       )
    data_2D[4] = (data[7],  0,          data[6],    0,          data[23],   0,          data[24],   0,          data[25])
    data_2D[5] = (0,        data[8],    0,          data[9],    0,          data[27],   0,          data[26],   0       )
    data_2D[6] = (data[11], 0,          data[10],   0,          data[15],   0,          data[28],   0,          data[29])
    data_2D[7] = (0,        0,          0,          data[12],   0,          data[30],   0,          0,          0       )
    data_2D[8] = (0,        0,          0,          data[13],   data[14],   data[31],   0,          0,          0       )
    return data_2D


def norm_dataset(dataset_1D):
    norm_dataset_1D = np.zeros([dataset_1D.shape[0], 32])
    for i in range(dataset_1D.shape[0]):
        norm_dataset_1D[i] = feature_normalize(dataset_1D[i])
    return norm_dataset_1D


def feature_normalize(data):
    mean = data[data.nonzero()].mean()
    sigma = data[data. nonzero ()].std()
    data_normalized = data
    data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
    return data_normalized


def dataset_1Dto2D(dataset_1D):
    dataset_2D = np.zeros([dataset_1D.shape[0],9,9])
    for i in range(dataset_1D.shape[0]):
        dataset_2D[i] = data_1Dto2D(dataset_1D[i])
    return dataset_2D


def norm_dataset_1Dto2D(dataset_1D):
    norm_dataset_2D = np.zeros([dataset_1D.shape[0], 9, 9])
    for i in range(dataset_1D.shape[0]):
        norm_dataset_2D[i] = feature_normalize(data_1Dto2D(dataset_1D[i]))
    return norm_dataset_2D


def windows(data, size):
    start = 0
    while ((start+size) < data.shape[0]):
        yield int(start), int(start + size)
        start += size

        
def segment_signal_without_transition(data,label,label_index,window_size):
    for (start, end) in windows(data, window_size):
        if((len(data[start:end]) == window_size)):
            if(start == 0):
                segments = data[start:end]
                segments = np.vstack([segments, data[start:end]])

#                 labels = np.array(label[label_index])
#                 labels = np.append(labels, np.array(label[label_index]))
            else:
                segments = np.vstack([segments, data[start:end]])
#                 labels = np.append(labels, np.array(label[label_index])) 
    return segments, np.array(label[label_index])


def apply_mixup(dataset_file,window_size,label,basemean):
    print("Processing",dataset_file,"..........")
    data_file_in = sio.loadmat(dataset_file)
    data_in = data_file_in["data"].transpose(0,2,1)
    #0 valence, 1 arousal, 2 dominance, 3 liking
    if label=="arousal":
        label=1
    elif label=="valence":
        label=0
    label_in= data_file_in["labels"][:,label]
    label_inter = np.empty([0]) 
    data_inter_E = np.empty([0, 60*window_size, 9, 9])
    data_inter_C = np.empty([0, 60*window_size, 32])
    trials = data_in.shape[0]


    for trial in range(0,trials):
        if basemean=="yes":
            base_signal = (data_in[trial,0:128,0:32]+data_in[trial,128:256,0:32]+data_in[trial,256:384,0:32])/3
        else:
            base_signal = 0
        data_E = data_in[trial,384:8064,0:32]
        # data_C = data_in[trial,384:8064,0:32]

        for i in range(0,60):
            data_E[i*128:(i+1)*128,0:32]=data_E[i*128:(i+1)*128,0:32]-base_signal
        label_index = trial

        data_E = norm_dataset(data_E)
        data_E, label = segment_signal_without_transition(data_E, label_in, label_index, window_size)
        # data_C = norm_dataset(data_C)
        # data_C, label = segment_signal_without_transition(data_C, label_in, label_index, window_size)

        data_E = dataset_1Dto2D(data_E)
        data_E = data_E.reshape (1, data_E.shape[0], 9, 9)

        # data_C = data_C.reshape(1, data_C.shape[0], 32)

        data_inter_E  = np.vstack([data_inter_E, data_E])
        # data_inter_C  = np.vstack([data_inter_C, data_C])
        # label_inter = np.append(label_inter, label)
    
    print("EEG data size:", data_inter_E.shape)
    # print("EEG-ContextNet data size:", data_inter_C.shape)
    # print("total label size:", label_inter.shape)
    

    return data_inter_E ,data_inter_C, label_inter

In [15]:
begin = time.time()
print("time begin:",time.localtime())

dataset_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/Feature_Extraction/"
basemean = "yes"     # yes/no (using baseline signals or not)

record_list = [task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))]
output_EEG_dir = output_dir+"Base_EEG/"+f"basemean_{basemean}"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in record_list:
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pkl"

        with open(output_data_cnn, "wb") as fp:
            pickle.dump(data_inter_E[trial],fp, protocol=4)
    end = time.time()
    print("end time:",time.localtime())
    print("time consuming:",(end-begin))

time begin: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=28, tm_sec=49, tm_wday=1, tm_yday=143, tm_isdst=0)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s03.mat ..........
EEG data size: (40, 7680, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=29, tm_sec=1, tm_wday=1, tm_yday=143, tm_isdst=0)
time consuming: 11.328668355941772
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s15.mat ..........
EEG data size: (40, 7680, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=29, tm_sec=11, tm_wday=1, tm_yday=143, tm_isdst=0)
time consuming: 22.037659883499146
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s09.mat ..........
EEG data size: (40, 7680, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=29, tm_sec=22, tm_wday=1, tm_yday=143, tm_isdst=0)
time co

In [16]:
begin = time.time()
print("time begin:",time.localtime())

dataset_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/Feature_Extraction/"
label_class = "arousal"     # arousal/valence
basemean = "no"     # yes/no (using baseline signals or not)

record_list = [task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))]
output_EEG_dir = output_dir+"Base_EEG/"+f"basemean_{basemean}"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in record_list:
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pkl"

        with open(output_data_cnn, "wb") as fp:
            pickle.dump(data_inter_E[trial],fp, protocol=4)
    end = time.time()
    print("end time:",time.localtime())
    print("time consuming:",(end-begin))

time begin: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=34, tm_sec=23, tm_wday=1, tm_yday=143, tm_isdst=0)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s03.mat ..........
EEG data size: (40, 7680, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=34, tm_sec=33, tm_wday=1, tm_yday=143, tm_isdst=0)
time consuming: 10.28294825553894
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s15.mat ..........
EEG data size: (40, 7680, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=34, tm_sec=43, tm_wday=1, tm_yday=143, tm_isdst=0)
time consuming: 20.573146104812622
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s09.mat ..........
EEG data size: (40, 7680, 9, 9)
end time: time.struct_time(tm_year=2023, tm_mon=5, tm_mday=23, tm_hour=17, tm_min=34, tm_sec=54, tm_wday=1, tm_yday=143, tm_isdst=0)
time co

In [28]:
dataset_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_label_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/EEG_label/"

record_list = sorted([task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))])

if os.path.isdir(output_label_dir)==False:
    os.makedirs(output_label_dir)

label_dataframe = pd.DataFrame(columns=["valence", "arousal"])

for record in record_list:
    file = os.path.join(dataset_dir,record)
    data_file_in = sio.loadmat(file)
    #0 valence, 1 arousal, 2 dominance, 3 liking
    label_in= data_file_in["labels"][:,0:2]
    label_dataframe = pd.concat([label_dataframe, pd.DataFrame(label_in, index=[record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}" for trial in range(40)], columns=["valence", "arousal"])])

output_label_save_path = output_label_dir + "EEG_label.csv"

label_dataframe.to_csv(output_label_save_path)

### save pt

In [26]:
dataset_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/Feature_Extraction/"
basemean = "yes"     # yes/no (using baseline signals or not)

record_list = sorted([task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))])
output_EEG_dir = output_dir+"Base_EEG/"+f"basemean_{basemean}_pt"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in tqdm(record_list):
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn_path = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pt"
        output_data_cnn_tensor = torch.tensor(data_inter_E[trial])

        torch.save(output_data_cnn_tensor, output_data_cnn_path)

  0%|          | 0/32 [00:00<?, ?it/s]

Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s01.mat ..........


  3%|▎         | 1/32 [00:10<05:24, 10.47s/it]

EEG data size: (40, 7680, 9, 9)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s02.mat ..........


  6%|▋         | 2/32 [00:20<05:14, 10.49s/it]

...

EEG data size: (40, 7680, 9, 9)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s31.mat ..........


 97%|█████████▋| 31/32 [05:22<00:10, 10.37s/it]

EEG data size: (40, 7680, 9, 9)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s32.mat ..........


100%|██████████| 32/32 [05:33<00:00, 10.41s/it]

EEG data size: (40, 7680, 9, 9)


In [27]:
dataset_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab"
window_size = 128
output_dir = "/home/aicv/Documents/2023_EEG/DEAP_Dataset/Feature_Extraction/"
basemean = "no"     # yes/no (using baseline signals or not)

record_list = sorted([task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))])
output_EEG_dir = output_dir+"Base_EEG/"+f"basemean_{basemean}_pt"+"/"
if os.path.isdir(output_EEG_dir)==False:
    os.makedirs(output_EEG_dir)

    
for record in tqdm(record_list):
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter = apply_mixup(file, window_size,label_class,basemean)
    trials = data_inter_E.shape[0]

    for trial in range(0,trials):
        output_data_cnn_path = output_EEG_dir+record.split(".")[0]+f"_trial{str(trial+1).zfill(2)}"+"_win_"+str(window_size)+".pt"
        output_data_cnn_tensor = torch.tensor(data_inter_E[trial])

        torch.save(output_data_cnn_tensor, output_data_cnn_path)

  0%|          | 0/32 [00:00<?, ?it/s]

Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s01.mat ..........


  3%|▎         | 1/32 [00:10<05:21, 10.37s/it]

EEG data size: (40, 7680, 9, 9)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s02.mat ..........


  6%|▋         | 2/32 [00:20<05:11, 10.37s/it]

...

EEG data size: (40, 7680, 9, 9)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s31.mat ..........


 97%|█████████▋| 31/32 [05:20<00:10, 10.36s/it]

EEG data size: (40, 7680, 9, 9)
Processing /home/aicv/Documents/2023_EEG/DEAP_Dataset/data_preprocessed_matlab/s32.mat ..........


100%|██████████| 32/32 [05:31<00:00, 10.35s/it]

EEG data size: (40, 7680, 9, 9)


### Reference code

In [ ]:
begin = time.time()
print("time begin:",time.localtime())
dataset_dir     =   "./data_preprocessed_matlab/"
window_size     =   128
output_dir      =   "./deap_data/"
label_class     =   "valence"     # arousal/valence
suffix          =   "yes"     # yes/no (using baseline signals or not)

record_list = [task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))]
output_dir = output_dir+suffix+"_"+label_class+"/"
if os.path.isdir(output_dir)==False:
    os.makedirs(output_dir)


for record in record_list:
    file = os.path.join(dataset_dir,record)
    data_inter_E,data_inter_C,label_inter,record = apply_mixup(file, window_size,label_class,suffix)
    output_data_cnn = output_dir+record+"_win_"+str(window_size)+"_emotion_dataset.pkl"
    output_data_rnn = output_dir+record+"_win_"+str(window_size)+"_context_dataset.pkl"
    output_label= output_dir+record+"_win_"+str(window_size)+"_labels.pkl"

    with open(output_data_cnn, "wb") as fp:
        pickle.dump(data_inter_E,fp, protocol=4)
    with open(output_data_rnn, "wb") as fp:
        pickle.dump(data_inter_C, fp, protocol=4)
    with open(output_label, "wb") as fp:
        pickle.dump(label_inter, fp)
    end = time.time()
    print("end time:",time.localtime())
    print("time consuming:",(end-begin))